# Opentron Protocol Simple Development Guide
Here we will present tips and tricks for developing new protocols and optimizing older protocols. There are essentially two main ways to run the robot. The first method is to use a jupyter notebook launched from the opentron app and the second is to use a python script with the opentron app. <br>
I would recommend starting new protocols/debugging old protocols using jupyter notebook for finer control over which step to run. When actually running a full protocol/experiment, python script would be the way to go for final testings.

# Presets: Defining Labware and Slot Location
Beginning a protocol, we first need a defined list of all the labware required. Note that opentron is pretty specific about its labware. Their api have specific api names (ex:'nest_96_wellplate_200ul_flat' for specific labwares that need to match perfectly for the protocol to run without error
* Note: If the plates/racks don't match exactly by brand, it should still be fine, you would just need to find the most similar labware in their labware library, and do some calibration to fit that particular labware.
* Link to the labware library can be found here: https://labware.opentrons.com/
* You also will want to assign a slot number for where the labware will be located.


# Calibration
Once we have the list of labware, we need to calibrate the robot before proceeding with protocol development. As of now, opentron doesn't support robot calibration through jupyter, so the best way is to use a dummy python protocol/script and calibrate it through the opentron app.

For example: <br>
If a 200ul 96 wellplate is need with 300ul tips we can load this dummy python protocol that simply loads all the equipment.

Note we won't be actually running the protocol. Before the run, the Opentron App will prompt you to calibrate the labware and save the calibration. There is no need to actually run the robot.

# Protocol Development
Now that all the labware have been properly calibrated, we can proceed with actually developing the protocol. Note that for cells in this notebook to actually move the robot, the jupyter notebook server must be set up through the Opentron App and not through your terminal. A error message will pop up if you try to run the below code through a server set up on your terminal.

In [2]:
import opentrons.execute

In [3]:
protocol = opentrons.execute.get_protocol_api('2.10') #Make sure this api version matches your robot's current software version.
protocol.home()

This is intended to run on a robot, and while it can connect to a smoothie via a usb/serial adapter unexpected things using gpios (such as smoothie reset or light management) will fail. If you are seeing this message and you are running on a robot, you need to set the RUNNING_ON_PI environmental variable to 1.
/Users/JeffreyWang/.opentrons/deck_calibration.json not found. Loading defaults
/Users/JeffreyWang/.opentrons/robot_settings.json not found. Loading defaults
Failed to initialize character device, will not be able to control gpios (lights, button, smoothiekill, smoothie reset). Only one connection can be made to the gpios at a time. If you need to control gpios, first stop the robot server with systemctl stop opentrons-robot-server. Until you restart the server with systemctl start opentrons-robot-server, you will be unable to control the robot using the Opentrons app.
Failed to initiate aionotify, cannot watch modules or door, likely because not running on linux
Motor driver cou

ThreadManagerException: Failed to create Managed Object

Note: Whenever get_protocol_api is called, the robot will return a new ProtocolContext and reset the state of the system.
Before moving the robot (for the first time), `protocol.home()` must be called.

The best way to develop your protocol is to modularize all the steps into functions, so that repeated/similar steps can be easily called (with some changes in the arguments).


## Load Labware and Instrument
First step is to load the labware and instruments.

In [ ]:
#Labware
tiprack = protocol.load_labware('opentrons_96_tiprack_300ul', '1') #'1' and '2' refer to the physical location on the robot deck
plate = protocol.load_labware('nest_96_wellplate_200ul_flat', '2')

# Instrument (pipettes)
p300 = protocol.load_instrument('p300_multi_gen2', 'right', tip_racks=[tiprack])

#Modules (magnet, temp (hot/cold plate), thermocycler)
magnetic_module = protocol.load_module('magnetic module gen2', '1') #Note we have gen2 module
magnetic_module.disengage()

Note: Plate wells are named A1, A2, A3.. for the first row of the plate and B1 for second row and so on...

## Step1: Example
You will want to test all the code to make sure the robot does exactly what you want before wrapping the steps in a functional step.

Place in function to represent in a step.

In [ ]:
def DNA_RNA_lysis_buffer_transfer(transfer_amount, mix_num, mix_amount, aspirate_speed, dispense_speed):
    global well_list
    global num_column
    global sample_column_list
    left_300_pipette.flow_rate.aspirate = aspirate_speed
    left_300_pipette.flow_rate.dispense = dispense_speed
    for x in range(num_column):
        load_tips()
        if x == 0:
            left_300_pipette.mix(mix_num, mix_amount, DNA_RNA_Lysis_Buffer_well)

        left_300_pipette.aspirate(transfer_amount, DNA_RNA_Lysis_Buffer_well)
        left_300_pipette.dispense(transfer_amount, reaction_plate[sample_column_list[x]])
        mixing(180, 5, x)
        left_300_pipette.move_to(reaction_plate[sample_column_list[x]].top(-5))
        protocol.delay(seconds=3)
        left_300_pipette.blow_out(reaction_plate[sample_column_list[x]].top(-5))
        discard_tips()

In [ ]:
#Sample call within the run function would like this.
protocol.comment(" ")
protocol.comment("DNA and RNA Lysis Buffer Transfer")
protocol.comment(" ")
DNA_RNA_lysis_buffer_transfer(200, 3, 100, 50, 100)

Repeat for each step.

# Compile Everything into a Single Protocol in a Python Script

Components:
1. Package required: `from opentrons import protocol_api, types`
2. Metadata dictionary describing the protocol and the current api version. Look into the Opentron app to see what api version to specifiy.
3. Main run function that encapsulates all the steps.

### Multiple Samples
We currently have pipettes with 8 multi-channels. Work with multiple samples, I would include a sample_num/col_num delineating how many columns of (8) samples you want the protocol to run. Once the logic for each protocl step is done you can simply loop through the number of columns.

# Test Mode:
One thing I like to do is include a test mode setting in the protocol to run water tests. This is mainly to conserve tips and labware.

In [ ]:
def load_tips():
    if test_mode:
        if tr_200[0].next_tip(num_tips=8):
            left_300_pipette.pick_up_tip()
        else:
            left_300_pipette.reset_tipracks() #Reset tiprack[0] to keep using.
            left_300_pipette.pick_up_tip()
    else:
        left_300_pipette.pick_up_tip()

In [ ]:
#To prevent unncessary wastes during testing we write a discard tip function
def discard_tips():
    if test_mode:
        left_300_pipette.move_to(liquid_waste.top(-2))
        protocol.delay(seconds=3)
        left_300_pipette.blow_out(liquid_waste.top(-2))
        left_300_pipette.return_tip()
    else:
        left_300_pipette.drop_tip()

# Useful Methods:

In [ ]:
def mixing(amount, rep, well_num, aspirate_speed=150, dispense_speed=300): #Default for robot is 150 and 300
    left_300_pipette.flow_rate.aspirate = aspirate_speed
    left_300_pipette.flow_rate.dispense = dispense_speed
    loc1 = reaction_plate[sample_column_list[well_num]].bottom().move(types.Point(x=1, y =0, z=.6)) #May need to change reaction_plate to work for your protocol.
    loc2 = reaction_plate[sample_column_list[well_num]].bottom().move(types.Point(x=1, y =0, z=5.5))
    left_300_pipette.aspirate(20, loc1)
    for x in range(rep):
        left_300_pipette.aspirate(amount, loc1)
        left_300_pipette.dispense(amount, loc2)
    left_300_pipette.dispense(20, loc2)
    left_300_pipette.flow_rate.aspirate = 50
    left_300_pipette.flow_rate.dispense = 100

### Opentron defaults:
Below are some defaults that are useful, copied from their documentation: https://docs.opentrons.com/v2/new_atomic_commands.html
<br> These are methods that will be used most often during protocol development.
#### Eliminating droplets
Often times tips will have droplet after dispensing. I would recommend using either blow out or touch tip to prevent any dripping.

In [ ]:
pipette.blow_out()            # blow out in current location

In [ ]:
pipette.touch_tip()            # touch tip within current location to the nearest wall of the plate/well
pipette.touch_tip(v_offset=-2) # touch tip 2mm below the top of the current location

#### Mix
Will mix but pipette won't move (i.e: up and down) while mixing. Hence the user defined mixing method, which may be needed when working with suspensions that may settle with time.

In [ ]:
pipette.mix(4, 100, plate.['A2']) 

#### Delay
Some protocols will require delay while a instrument module is engaged (ex: using a heatblock or a magnet).

In [ ]:
protocol.delay(seconds=2)             # pause for 2 seconds
protocol.delay(minutes=5)             # pause for 5 minutes

#### User-defined pauses
Use this function when you need an undefined pause time to either switch out lab equipment/samples or do any benchwork. To continue with protocol clikc resume on the Opentron App.

In [ ]:
protocol.pause('Switch plates')